In [ ]:
from sklearn.pipeline import Pipeline
from sklearn.model_selection import RandomizedSearchCV
from sklearn.svm import SVC
from sklearn.metrics import classification_report

# Configurar el Pipeline
svc_pipeline = Pipeline(steps=[('classifier', SVC(probability=True, random_state=seed))])

# Definir el espacio de hiperparámetros
param_grid = {
    'classifier__C': [0.1, 1, 10, 100],                 # Penalización del margen
    'classifier__kernel': ['linear', 'poly', 'rbf'],   # Tipo de kernel
    'classifier__gamma': ['scale', 'auto', 0.1, 0.01], # Coeficiente del kernel
    'classifier__degree': [2, 3, 4]                    # Grado del polinomio (solo para kernel poly)
}

# Configurar RandomizedSearchCV
CV = RandomizedSearchCV(
    estimator=svc_pipeline,
    param_distributions=param_grid,
    n_iter=10,  # Número de combinaciones a probar
    cv=10,  # Validación cruzada
    scoring='accuracy',  # Métrica de evaluación
    random_state=seed,  # Semilla para reproducibilidad
    n_jobs=-1,  # Usar todos los núcleos disponibles
    verbose=2
)

# Entrenar el modelo
CV.fit(X_resampled, y_resampled)

# Predecir en el conjunto de validación
y_pred_proba = CV.predict_proba(X_val)[:, 1]
threshold = 0.3  # Prueba diferentes valores de umbral
y_pred = (y_pred_proba >= threshold).astype(int)

# Imprimir los mejores parámetros y el mejor score
print("Mejores parámetros:", CV.best_params_)
print("Mejor score:", CV.best_score_)

# Reporte de clasificación
print(classification_report(y_val, y_pred))
